# Logistic Regression with daru and statsample-glm

In this notebook we'll see with some examples how the probability of a given outcome can be predicted with logistic regression using daru and statsample-glm.

In [1]:
require 'daru'
require 'statsample-glm'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"http://d3js.org/d3.v3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t

true

For this notebook, we will utilize [this dataset](http://www.ats.ucla.edu/stat/data/binary.csv) denoting whether students got admission for a graduate degree program depending on their GRE scores, GPA and rank of the institute they did an undergraduate degree in (ranked from 1 to 4).

It should be noted that statsample-glm does not yet support categorical data so the ranks will be treated as continuos.

In [2]:
df = Daru::DataFrame.from_csv "binary.csv"
df.vectors = Daru::Index.new([:admit, :gpa, :gre, :rank])
df

,admit,gpa,gre,rank
0,0,3.61,380,3
1,1,3.67,660,3
2,1,4,800,1
3,1,3.19,640,4
4,0,2.93,520,4
5,1,3,760,2
6,1,2.98,560,1
7,0,3.08,400,2
8,1,3.39,540,3
9,0,3.92,700,2


Use the `Statsampel::GLM.compute` method for logisitic regression analysis.

The first method in the `compute` function is the DataFrame object, followed by the Vector that is to be the dependent variable, and then the method to be used for the link function. Can be :logit, :probit, :poisson or :normal.

The `coefficients` method calculates the coefficients of the GLM and returns them as a Hash.

In [3]:
glm = Statsample::GLM::compute df, :admit, :logistic, constant: 1
c = glm.coefficients :hash

{:gpa=>0.777013573719857, :gre=>0.0022939595044433273, :rank=>-0.5600313868499897, :constant=>-3.4495483976684773}

The logistic regression coefficients give the change in the log odds of the outcome for a one unit increase in the predictor variable.

Therefore, to interpret each of the above co-efficients:
* For every one unit change in gre, the log odds of admission (versus non-admission) increases by **0.002**.
* For a one unit increase in gpa, the log odds of being admitted to graduate school increases by **0.777**.
* For every increase in the rank number of the institute (aka decrease in quality of the institute), the log odds of being admitted to graduate school increase by **-0.56**.

Log odds become a little difficult to interpret, so we'll exponentiate each of the co-efficients so that each co-efficient can be interpreted as an odds-ratio.

In [4]:
Daru::Vector.new(c).exp # Calling `#exp` on Daru::Vector exponentiates each element of the Vector.

#<Daru::Vector:77641600 @name = nil @size = 4 >
                                    nil
                gpa   2.174967177712439
                gre  1.0022965926425997
               rank   0.571191135676971
           constant 0.03175997601913591

We can now compute the probability of gaining admission into a graduate college based on the rank of the undergraduate college, by keeping the GRE score and GPA constant.

As you can see in the result below, the `rankp` Vector shows the probability of admission based on the rank. The person from the most highly rated undergrad school (rank 1) has a probability of **0.49** of getting admitted into graduate school.

In [5]:
e = Math::E
new_data = Daru::DataFrame.new({
  gre: [df[:gre].mean]*4,
  gpa: [df[:gpa].mean]*4,
  rank: df[:rank].factors
  })

new_data[:rankp] = new_data.collect(:row) do |x|
  1 / (1 + e ** -(c[:constant]  + x[:gre] * c[:gre] + x[:gpa] * c[:gpa] + x[:rank] * c[:rank]))
end

new_data.sort! [:rank]

,gpa,gre,rank,rankp
1,3.3899000000000017,587.7,1,0.4931450619837156
3,3.3899000000000017,587.7,2,0.357219500353945
0,3.3899000000000017,587.7,3,0.240948896129993
2,3.3899000000000017,587.7,4,0.1534862275970381


To demonstrate with another example, lets create a hypothetical dataset consisting of the body weight of 20 people and whether they survived or not.

For this example we will just assume that people with less body weight have lesser chances of survival.

In [6]:
require 'distribution'

# Create a normally distributed Vector with mean 30 and standard deviation 2
rng = Distribution::Normal.rng(30,2)
body_weight = Daru::Vector.new(20.times.map { rng.call }.sort)

# Populate chances of survival, assume that people with less body weight on average
# are less likely to survive.
survive = Daru::Vector.new [0,0,0,0,0,1,0,1,0,0,1,1,0,1,1,1,0,1,1,1]

df = Daru::DataFrame.new({
  body_weight: body_weight,
  survive: survive
})

,body_weight,survive
0,24.908838843051175,0
1,26.39702077725302,0
2,26.537503104355725,0
3,27.696995615896935,0
4,27.819742173047572,0
5,28.013650374913873,1
6,29.07404001096605,0
7,29.437133273453572,1
8,29.46564196358177,0
9,29.765654520629372,0


Compute the logistic regression co-efficients.

In [7]:
glm    = Statsample::GLM.compute df, :survive, :logistic, constant: 1
coeffs = glm.coefficients :hash

{:body_weight=>0.8476692915502841, :constant=>-25.069555663225014}

Based on the coefficients, we compute the predicted probabilities for each number in the Vector :body_weight and store them in another Vector called `:survive_pred`.

In [8]:
e = Math::E
df[:survive_pred] = df[:body_weight].map { |x| 1 / (1 + e ** -(coeffs[:constant] + x*coeffs[:body_weight])) }
df

,body_weight,survive,survive_pred
0,24.908838843051175,0,0.018796708581676695
1,26.39702077725302,0,0.06335136542459532
2,26.537503104355725,0,0.07079577610436874
3,27.696995615896935,0,0.16915005150646575
4,27.819742173047572,0,0.18428004426502645
5,28.013650374913873,1,0.21027900032655508
6,29.07404001096605,0,0.39546799202592625
7,29.437133273453572,1,0.4708825435197856
8,29.46564196358177,0,0.47690748653921533
9,29.765654520629372,0,0.5403807655136829


The above results can then be plotted using the `plot` function.

The curve looks is an ideal logit regression curve.

In [9]:
df.plot type: [:scatter,:line], x: [:body_weight]*2, y: [:survive_pred]*2 do |plot, diagram|
  plot.x_label "Body Weight"
  plot.y_label "Probability of Survival"
end

#<Nyaplot::Frame:0xac5cdbc @properties={:panes=>[#<Nyaplot::Plot:0xac7ed90 @properties={:diagrams=>[#<Nyaplot::Diagram:0xac5d460 @properties={:type=>:scatter, :options=>{:x=>:body_weight, :y=>:survive_pred}, :data=>"f21968c5-bb74-44e9-a4b5-e00d98b8a489"}, @xrange=[24.908838843051175, 34.91453329996092], @yrange=[0.018796708581676695, 0.9892964867667766]>, #<Nyaplot::Diagram:0xac5d104 @properties={:type=>:line, :options=>{:x=>:body_weight, :y=>:survive_pred}, :data=>"f21968c5-bb74-44e9-a4b5-e00d98b8a489"}, @xrange=[24.908838843051175, 34.91453329996092], @yrange=[0.018796708581676695, 0.9892964867667766]>], :options=>{:x_label=>"Body Weight", :y_label=>"Probability of Survival", :zoom=>true, :width=>700, :xrange=>[24.908838843051175, 34.91453329996092], :yrange=>[0.018796708581676695, 0.9892964867667766]}}>], :data=>{"f21968c5-bb74-44e9-a4b5-e00d98b8a489"=>#<Nyaplot::DataFrame:0xac5dbf4 @name="f21968c5-bb74-44e9-a4b5-e00d98b8a489", @rows=[{:body_weight=>24.908838843051175, :survive=>0, :survive_pred=>0.018796708581676695}, {:body_weight=>26.39702077725302, :survive=>0, :survive_pred=>0.06335136542459532}, {:body_weight=>26.537503104355725, :survive=>0, :survive_pred=>0.07079577610436874}, {:body_weight=>27.696995615896935, :survive=>0, :survive_pred=>0.16915005150646575}, {:body_weight=>27.819742173047572, :survive=>0, :survive_pred=>0.18428004426502645}, {:body_weight=>28.013650374913873, :survive=>1, :survive_pred=>0.21027900032655508}, {:body_weight=>29.07404001096605, :survive=>0, :survive_pred=>0.39546799202592625}, {:body_weight=>29.437133273453572, :survive=>1, :survive_pred=>0.4708825435197856}, {:body_weight=>29.46564196358177, :survive=>0, :survive_pred=>0.47690748653921533}, {:body_weight=>29.765654520629372, :survive=>0, :survive_pred=>0.5403807655136829}, {:body_weight=>29.879156612383376, :survive=>1, :survive_pred=>0.5641661861299625}, {:body_weight=>29.880823486329877, :survive=>1, :survive_pred=>0.5645135764700635}, {:body_weight=>30.160725313561752, :survive=>0, :survive_pred=>0.6216987926155648}, {:body_weight=>30.298299559539867, :survive=>1, :survive_pred=>0.6487120001947825}, {:body_weight=>30.67811574712452, :survive=>1, :survive_pred=>0.7181575967888023}, {:body_weight=>30.9289957867363, :survive=>1, :survive_pred=>0.7591462024443916}, {:body_weight=>31.335058165003584, :survive=>0, :survive_pred=>0.8164098835252157}, {:body_weight=>31.62700650952587, :survive=>1, :survive_pred=>0.8506472456646877}, {:body_weight=>31.785864486685178, :survive=>1, :survive_pred=>0.8669602955924562}, {:body_weight=>34.91453329996092, :survive=>1, :survive_pred=>0.9892964867667766}]>}, :extension=>[]}>